# Event - Async Execution with Lifecycle Tracking

The `Event` class provides **async execution with observable lifecycle states**. Events track execution status, capture responses/errors, measure duration, and support retry patterns.

**Key Features:**
- **Lifecycle**: `PENDING` → `PROCESSING` → `COMPLETED`/`FAILED`/`CANCELLED`
- **Idempotency**: Multiple `invoke()` calls return cached result
- **Observability**: Status, duration, response, error, retryable flag
- **Retry Pattern**: `as_fresh_event()` creates fresh instance with reset state
- **Timeout Support**: Optional timeout with `lionprideTimeoutError`

In [ ]:
# Imports
import asyncio

from lionpride.core import Event
from lionpride.types import Unset


# Define test events
class SimpleEvent(Event):
    """Event that returns a value."""

    return_value: object = None

    async def _invoke(self):
        return self.return_value


class FailingEvent(Event):
    """Event that raises an exception."""

    error_msg: str = "Test error"

    async def _invoke(self):
        raise ValueError(self.error_msg)


class SlowEvent(Event):
    """Event with configurable delay."""

    delay: float = 0.1
    return_value: object = "completed"

    async def _invoke(self):
        await asyncio.sleep(self.delay)
        return self.return_value

## 1. Construction + Basic Invocation

In [ ]:
# Create event
event = SimpleEvent(return_value=42)

# Initial state: PENDING, no response yet
print(f"Status: {event.status}")
print(f"Response: {event.execution.response}")  # Unset sentinel

# Execute
result = await event.invoke()

# After execution: COMPLETED, response captured
print(f"\nResult: {result}")
print(f"Status: {event.status}")
print(f"Response: {event.execution.response}")
print(f"Duration: {event.execution.duration:.4f}s")

Status: pending
Response: Unset

Result: None
Status: completed
Response: 42
Duration: 0.0000s


## 2. Status Lifecycle Transitions

In [ ]:
# Track status transitions
status_history = []


class StatusTrackingEvent(Event):
    async def _invoke(self):
        status_history.append(self.status.value)
        await asyncio.sleep(0.01)
        return "done"


event = StatusTrackingEvent()
status_history.append(event.status.value)

result = await event.invoke()
status_history.append(event.status.value)

print("Lifecycle: " + " → ".join(status_history))
# Output: pending → processing → completed

Lifecycle: pending → processing → completed


## 3. Success vs Failure Handling

In [ ]:
# Success case
success = SimpleEvent(return_value="success!")
result = await success.invoke()
print(f"Success: {result}")
print(f"  Status: {success.status}")
print(f"  Error: {success.execution.error}")
print(f"  Retryable: {success.execution.retryable}")

# Failure case (exception caught, stored in execution)
failure = FailingEvent(error_msg="Something broke")
result = await failure.invoke()
print(f"\nFailure: {result}")  # Returns None on failure
print(f"  Status: {failure.status}")
print(f"  Error: {failure.execution.error}")
print(f"  Retryable: {failure.execution.retryable}")  # Unknown exceptions default to True

Success: None
  Status: completed
  Error: None
  Retryable: False

Failure: None
  Status: failed
  Error: Something broke
  Retryable: True


## 4. Timeout Handling

In [ ]:
# Event with timeout - completes in time
fast = SlowEvent(delay=0.01, return_value="fast", timeout=1.0)
result = await fast.invoke()
print(f"Fast event: {result}")
print(f"  Status: {fast.status}")

# Event with timeout - exceeds timeout
slow = SlowEvent(delay=5.0, timeout=0.05)
result = await slow.invoke()
print(f"\nSlow event: {result}")  # Returns None on timeout
print(f"  Status: {slow.status}")  # CANCELLED
print(f"  Error: {type(slow.execution.error).__name__}")  # lionprideTimeoutError
print(f"  Message: {slow.execution.error}")
print(f"  Retryable: {slow.execution.retryable}")  # Timeouts are retryable

Fast event: None
  Status: completed

Slow event: None
  Status: cancelled
  Error: TimeoutError
  Message: Operation timed out after 0.05s
  Retryable: True


## 5. Idempotency - Cached Results

In [ ]:
# Track invocation count
counter = 0


class CountingEvent(Event):
    async def _invoke(self):
        global counter
        counter += 1
        return f"result_{counter}"


event = CountingEvent()

# First call - executes _invoke()
result1 = await event.invoke()
print(f"Call 1: {result1}, counter={counter}")

# Second call - returns cached result WITHOUT executing _invoke()
result2 = await event.invoke()
print(f"Call 2: {result2}, counter={counter}")  # Same result, counter unchanged

# Third call - still cached
result3 = await event.invoke()
print(f"Call 3: {result3}, counter={counter}")  # Idempotent!

# Multiple concurrent invocations also return same cached result
results = await asyncio.gather(event.invoke(), event.invoke(), event.invoke())
print(f"\nConcurrent calls: {results}, counter={counter}")  # All same, no re-execution

Call 1: None, counter=1
Call 2: None, counter=1
Call 3: None, counter=1

Concurrent calls: [None, None, None], counter=1


## 6. Retry Pattern with `as_fresh_event()`

In [ ]:
# Original event fails
original = FailingEvent(error_msg="Network timeout")
original.metadata["attempt"] = 1

result = await original.invoke()
print(f"Original attempt {original.metadata['attempt']}: {original.status}")
print(f"  Error: {original.execution.error}")
print(f"  Retryable: {original.execution.retryable}")

# Check if retryable and create fresh event
if original.execution.retryable:
    fresh = original.as_fresh_event(copy_meta=True)
    fresh.metadata["attempt"] = 2

    print(f"\nFresh event (attempt {fresh.metadata['attempt']})")
    print(f"  New ID: {fresh.id != original.id}")
    print(f"  Status reset: {fresh.status}")  # PENDING
    print(f"  Response reset: {fresh.execution.response}")  # Unset
    print(f"  Original tracking: {fresh.metadata.get('original', {}).get('id')}")

    # Fresh event can be executed independently
    result = await fresh.invoke()
    print(f"  Fresh result: {fresh.status}")

Original attempt 1: failed
  Error: Network timeout
  Retryable: True

Fresh event (attempt 2)
  New ID: True
  Status reset: pending
  Response reset: Unset
  Original tracking: 66cd74f5-e847-47d0-a132-e410c22c9eb9
  Fresh result: failed


## 7. ExceptionGroup Support

In [ ]:
# Event that raises multiple errors
class MultiErrorEvent(Event):
    async def _invoke(self):
        errors = [
            ValueError("validation error"),
            TypeError("type mismatch"),
            KeyError("missing key"),
        ]
        raise ExceptionGroup("multiple errors", errors)


event = MultiErrorEvent()
result = await event.invoke()

print(f"Status: {event.status}")
print(f"Error type: {type(event.execution.error).__name__}")
print(f"Error count: {len(event.execution.error.exceptions)}")
print(f"Retryable: {event.execution.retryable}")  # True only if ALL errors retryable

# Serialization handles ExceptionGroup
serialized = event.execution.to_dict()
print(f"\nSerialized error: {serialized['error']['error']}")
print(f"Nested exceptions: {len(serialized['error']['exceptions'])}")
for i, exc in enumerate(serialized["error"]["exceptions"]):
    print(f"  {i + 1}. {exc['error']}: {exc['message']}")

Status: failed
Error type: ExceptionGroup
Error count: 3
Retryable: True

Serialized error: ExceptionGroup
Nested exceptions: 3
  1. ValueError: validation error
  2. TypeError: type mismatch
  3. KeyError: 'missing key'


## 8. Sentinel Handling (Unset vs None)

In [ ]:
# Unset: Event never executed or failed (no response available)
pending = SimpleEvent()
print(f"Pending event response: {pending.execution.response}")
print(f"Is Unset: {pending.execution.response is Unset}")

# None: Event completed successfully with null value
none_event = SimpleEvent(return_value=None)
result = await none_event.invoke()
print(f"\nNone event result: {result}")
print(f"Response is None: {none_event.execution.response is None}")
print(f"Status: {none_event.status}")  # COMPLETED

# Both serialize to None, but status provides context
print("\nSerialization:")
print(f"  Pending: {pending.execution.to_dict()['response']} (status={pending.status.value})")
print(f"  None: {none_event.execution.to_dict()['response']} (status={none_event.status.value})")

Pending event response: Unset
Is Unset: True

None event result: None
Response is None: True
Status: completed

Serialization:
  Pending: None (status=pending)
  None: None (status=completed)


## Summary

The `Event` system provides **production-grade async execution** with:

1. **Lifecycle Management**: Observable state transitions (PENDING → PROCESSING → terminal)
2. **Idempotency**: Concurrent/repeated invocations return cached result
3. **Error Capture**: Exceptions stored in execution state (no propagation)
4. **Timeout Support**: Optional timeout with retryable `lionprideTimeoutError`
5. **Retry Pattern**: `as_fresh_event()` creates fresh instance for retries
6. **Observability**: Duration, response, error, retryable flag tracked
7. **ExceptionGroup**: Parallel failure aggregation with conservative retryability
8. **Sentinel Semantics**: Distinguish "no value" (Unset) from "null value" (None)

**When to Use:**
- Trackable async operations (API calls, DB queries, computations)
- Retry logic with `as_fresh_event()` + `retryable` flag
- Distributed tracing via `Execution` serialization
- Timeout-aware operations

**Next Steps:**
- Subclass `Event` and implement `_invoke()` for custom operations
- Use `execution.to_dict()` for logging/monitoring
- Build retry strategies with `as_fresh_event()` + retryable checks